# How to use NFM for your own data and task
This notebook demonstrates how to implment one's own NFM on own data. 

Note that this notebook is not to instruct on how to use the pre-coded packages (NFM + predictor) in each task sub-folder, but rather to demo how to use the backbone NFM in the model folder. 


## Basic setup  

In [ ]:
from torch.backends import cudnn
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from model.NFM_backbone import NFM_general
from utils.vars_ import HyperVariables

cudnn.benchmark = True
# Seed
np.random.seed(88)
torch.manual_seed(88)

# Generate "mini-batch" input data (replace it with your own dataset and dataloader) 
train_x_mb = torch.randn((32, 720, 10))

# Generate mini-batch sample target data (we will show different cases)

## True label for classification 
train_y_label = torch.int(0, 10, (32,))
target_lenth_label = 0

## True prediction for Forecasting (horizon = 180)
train_y_horizon = torch.randn((32, 180, 10))
target_lenth_horizon = 180


## Setup arguments and NFM instantiation

In [ ]:
# Model arguments (examplified for classification setup)

N = 720 # input length
L = target_lenth_horizon # or "target_lenth_label" for classification

Fx = N # sampling rate of input time series. Note that we always assume Tx = 1
Fy = Fx # sampling rate of output latent variable (note that Fy is not always the same as Fx -> see anomaly detection in our main work)

training_T_F = [Fy, Fx, L, N] # [output sr, input sr, target length, input length] -> for classification, you only need to change L to 0 or target_lenth_label
testing_T_F = [Fy, Fx, L, N] # we use this setup at testing time

hypervars = HyperVariables(sets_in_training = training_T_F,
                            sets_in_testing = testing_T_F,
                            C_ = 10,
                            multivariate = False, # False makes NFM chennel-independent
                            
                            # Mixing block
                            filter_type = "INFF",
                            hidden_dim = 32,
                            inff_siren_hidden = 32,
                            inff_siren_omega = 30,
                            layer_num = 1, # number of mixing blocks
                            
                            lft = True,
                            lft_siren_dim_in = 32,
                            lft_siren_hidden = 32,
                            lft_siren_omega = 30)

# Construct NFM 
NFM_backbone = NFM_general(hypervars)

# Prediction head (output dim is 1 for channel independence)
predictor = nn.Linear(32, 1) 

## Move NFM_backbone and predictor to a GPU if one is available and no need to move hypervars

## Training

In [ ]:
# Optimizer (add schedulers, decay, etc. as needed)
opt = torch.optim.Adam(NFM_backbone.parameters(), lr = 0.0001)
# Criterion
criterion = nn.MSELoss()

# Training 

hypervars.training_set() # this NFM to act on "training_T_F" setup
NFM_backbone.train()
for epoch in range(10):
     opt.zero_grad()
     train_x_mb = self.hyper_vars.input_(train_x_mb) # This rearranges the minibatch according to the channel independence.

     # InstanceNorm for forecasting (1)
     train_x_mb_mean = torch.mean(train_x_mb, dim=1, keepdim=True)
     train_x_mb_std= torch.sqrt(torch.var(train_x_mb, dim=1, keepdim=True)+ 1e-5)
     train_x_mb = train_x_mb - train_x_mb_mean
     train_x_mb = train_x_mb / train_x_mb_std
     
     # (2)
     z = NFM_backbone(train_x_mb) the output length is (N+L) !!
     y = predictor(z)

     # reverse instanceNorm (3)
     y = y * train_x_mb_std + train_x_mb_mean
     y, y_freq = self.hyper_vars.output_(y) # This rearranges the output minibatch according to the channel independence.

     # compute loss (4)
     fullspan_loss = nn.MSELoss(y, torch.cat((train_x_mb, train_y_horizon), dim = 1).detach() )
     fullspan_loss.backward()
     self.opt.step()

# Same for classification setup except that (1) and (3) are not necessary

## Testing at different sampling rate ($m_f \neq 1$)
Testing the trained NFM on the input time series sampled at different rate is easy. 

You can simply do this by setting a new input sampling rate and input length, and let NFM to work on this set of arguments. 

In [ ]:
# Testing-time inputs sampled at half the input sampling rate
test_x = torch.randn((32, 360, 10)) # N is 360 (downsampled) and so Fx = 360
test_y_horizon = torch.randn((32, 180, 10)) # no change in target prediction

N_test = 360
Fx_test = N

# Set testing_T_F again
testing_T_F = [Fy, Fx_test, L, N_test]
hypervars.sets_in_testing = testing_T_F

# Apply to NFM
hypervars.testing_set()

# Inference
test_x = self.hyper_vars.input_(test_x)

## InstanceNorm for forecasting (1)
test_x_mean = torch.mean(test_x, dim=1, keepdim=True)
test_x_std= torch.sqrt(torch.var(test_x, dim=1, keepdim=True)+ 1e-5)
test_x = test_x - test_x_mean
test_x = test_x / test_x_std

## (2)
test_z = NFM(test_x)
test_y = predictor(test_z)

## Reverse instanceNorm (3)
test_y = test_y * test_x_std + test_x_mean
y, y_freq = self.hyper_vars.output_(test_y) # This rearranges the output minibatch according to the channel independence.

y_horizon = y[:,-L:,:]

# Same for classification setup except that (1) and (3) are not necessary.